In [1]:
import os
import sys
# import pyshp

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

PROJ_ROOT = os.path.join(os.pardir)

In [3]:
def read_shapefile(shp_path):
	"""
	Read a shapefile into a Pandas dataframe with a 'coords' column holding
	the geometry information. This uses the pyshp package
	"""
	import shapefile

	#read file, parse out the records and shapes
	sf = shapefile.Reader(shp_path)
	fields = [x[0] for x in sf.fields][1:]
	records = [list(i) for i in sf.records()]
	shps = [s.points for s in sf.shapes()]

	#write into a dataframe
	df = pd.DataFrame(columns=fields, data=records)
	df = df.assign(coords=shps)

	return df

In [4]:
places_path = os.path.join(PROJ_ROOT, 
                         'data',
                         'interim',
                         '500_places_ct_filtered_2.csv')
census_path = os.path.join(PROJ_ROOT, 
                         'data',
                         'raw',
                         'chicago census tract.csv')

und_death_path = os.path.join(PROJ_ROOT, 
                         'data',
                         'processed',
                         'clean_underlyingcauseofdeath.csv')

mrfei_path = os.path.join(PROJ_ROOT, 
                         'data',
                         'raw',
                         '2_16_mrfei_data_table.xls')

boundary_path = os.path.join(PROJ_ROOT, 
                         'data',
                         'raw',
                             'census_tract_shape_file',
                         'cb_2018_17_tract_500k.shp')

fara_path = os.path.join(PROJ_ROOT, 
                         'data',
                         'interim',
                         'food_access_atlas.csv')

# census = pd.read_csv(census_path)

In [6]:
fara_df = pd.read_csv(fara_path)
census_df = pd.read_csv(census_path)
places_df = pd.read_csv(places_path)
und_death_df = pd.read_csv(und_death_path)
mrfei_df = pd.read_excel(mrfei_path)
boundary_df = read_shapefile(boundary_path)

In [7]:
fara_df.shape

(797, 93)

In [13]:
data_frames={'fara_df':fara_df,'census_df':census_df,
             'places_df':places_df,'und_death_df':und_death_df,
             'mrfei_df':mrfei_df, 'boundary_df':boundary_df}

for df in data_frames.values():
    cols = df.columns
    cols = [x.lower() for x in cols]
    df.columns = cols

In [14]:
for title,df in data_frames.items():
    print(title,'\n',df.shape,'\n\n')

fara_df 
 (797, 93) 


census_df 
 (801, 10) 


places_df 
 (797, 29) 


und_death_df 
 (6501, 9) 


mrfei_df 
 (65345, 3) 


boundary_df 
 (3121, 10) 




### Merge places and food access dataframes

In [15]:
places_df.head(1)

,locationname,all teeth lost among adults aged >=65 years,arthritis among adults aged >=18 years,binge drinking among adults aged >=18 years,cancer (excluding skin cancer) among adults aged >=18 years,cervical cancer screening among adult women aged 21-65 years,cholesterol screening among adults aged >=18 years,chronic kidney disease among adults aged >=18 years,chronic obstructive pulmonary disease among adults aged >=18 years,coronary heart disease among adults aged >=18 years,...,no leisure-time physical activity among adults aged >=18 years,obesity among adults aged >=18 years,"older adult men aged >=65 years who are up to date on a core set of clinical preventive services: flu shot past year, ppv shot ever, colorectal cancer screening","older adult women aged >=65 years who are up to date on a core set of clinical preventive services: flu shot past year, ppv shot ever, colorectal cancer screening, and mammogram past 2 years",physical health not good for >=14 days among adults aged >=18 years,sleeping less than 7 hours among adults aged >=18 years,stroke among adults aged >=18 years,taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure,visits to dentist or dental clinic among adults aged >=18 years,visits to doctor for routine checkup within the past year among adults aged >=18 years
0,17031010100,12.5,20.2,20.4,4.7,82.9,82.2,2.8,6.0,4.8,...,24.4,34.9,26.8,25.4,11.9,39.8,3.2,70.3,60.4,76.0


In [16]:
fara_df.head(1)

,censustract,state,county,urban,pop2010,ohu2010,groupquartersflag,numgqtrs,pctgqtrs,lilatracts_1and10,...,tractseniors,tractwhite,tractblack,tractasian,tractnhopi,tractaian,tractomultir,tracthispanic,tracthunv,tractsnap
0,17031010100,Illinois,Cook County,1,4854,2302,0,218.0,4.49,0,...,277.0,1810.0,2437.0,150.0,1.0,33.0,423.0,616.0,1162.0,433.0


In [17]:
full_df = pd.merge(fara_df, places_df, left_on='censustract',right_on='locationname')

In [18]:
full_df.shape

(797, 122)

The resulting dataframe is the same amount of rows as before

### Add the mrfei dataframe

In [19]:
mrfei_df.head(1)

,state,fips,mrfei
0,AL,1001020100,16.6667


In [20]:
mrfei_df=mrfei_df.fillna(0)
mrfei_df['str_fips'] = mrfei_df.fips.astype(str)

In [21]:
full_df.head(1)

,censustract,state,county,urban,pop2010,ohu2010,groupquartersflag,numgqtrs,pctgqtrs,lilatracts_1and10,...,no leisure-time physical activity among adults aged >=18 years,obesity among adults aged >=18 years,"older adult men aged >=65 years who are up to date on a core set of clinical preventive services: flu shot past year, ppv shot ever, colorectal cancer screening","older adult women aged >=65 years who are up to date on a core set of clinical preventive services: flu shot past year, ppv shot ever, colorectal cancer screening, and mammogram past 2 years",physical health not good for >=14 days among adults aged >=18 years,sleeping less than 7 hours among adults aged >=18 years,stroke among adults aged >=18 years,taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure,visits to dentist or dental clinic among adults aged >=18 years,visits to doctor for routine checkup within the past year among adults aged >=18 years
0,17031010100,Illinois,Cook County,1,4854,2302,0,218.0,4.49,0,...,24.4,34.9,26.8,25.4,11.9,39.8,3.2,70.3,60.4,76.0


In [15]:
pd.merge(full_df, mrfei_df, left_on='censustract',right_on='fips').shape

(556, 126)

In [16]:
full_data = pd.merge(full_df, mrfei_df, left_on='censustract',right_on='fips',how='left')

In [17]:
full_data.shape

(797, 126)

In [18]:
full_data.head()

,censustract,state_x,county,urban,pop2010,ohu2010,groupquartersflag,numgqtrs,pctgqtrs,lilatracts_1and10,...,physical health not good for >=14 days among adults aged >=18 years,sleeping less than 7 hours among adults aged >=18 years,stroke among adults aged >=18 years,taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure,visits to dentist or dental clinic among adults aged >=18 years,visits to doctor for routine checkup within the past year among adults aged >=18 years,state_y,fips,mrfei,str_fips
0,17031010100,Illinois,Cook County,1,4854,2302,0,218.0,4.49,0,...,11.9,39.8,3.2,70.3,60.4,76.0,IL,1.703101e+10,7.69231,17031010100
1,17031010201,Illinois,Cook County,1,6450,2463,0,163.0,2.53,0,...,14.0,41.2,3.3,67.5,52.7,73.4,NaN,NaN,NaN,NaN
2,17031010202,Illinois,Cook County,1,2818,1115,0,315.0,11.18,0,...,13.6,39.1,3.8,72.4,57.3,75.0,NaN,NaN,NaN,NaN
3,17031010300,Illinois,Cook County,1,6236,2826,0,791.0,12.68,0,...,12.0,36.6,3.5,73.3,62.8,75.4,IL,1.703101e+10,10.2564,17031010300
4,17031010400,Illinois,Cook County,1,5042,2098,0,1349.0,26.76,0,...,8.2,34.7,1.7,62.2,68.7,71.8,IL,1.703101e+10,8.16327,17031010400


In [22]:
mrfei_df[mrfei_df.str_fips.str.startswith('17031010')]

,state,fips,mrfei,str_fips
17767,IL,17031010100,7.69231,17031010100
17768,IL,17031010200,7.27273,17031010200
17769,IL,17031010300,10.2564,17031010300
17770,IL,17031010400,8.16327,17031010400
17771,IL,17031010500,6.66667,17031010500
17772,IL,17031010600,7.24638,17031010600
17773,IL,17031010700,8.47458,17031010700
17774,IL,17031010800,8.51064,17031010800
17775,IL,17031010900,6.52174,17031010900


The mrfei dataset doesn't have all of the census tracts. It appears as if for some tracts, the values are maybe aggregated?

### add boudnary data

In [20]:
boundary_df.head(1)

,statefp,countyfp,tractce,affgeoid,geoid,name,lsad,aland,awater,coords
0,17,031,843800,1400000US17031843800,17031843800,8438,CT,1309516,0,"[(-87.645538, 41.808864), (-87.64067899999999,..."


In [21]:
boundary_df.shape

(3121, 10)

In [22]:
full_data['locationname'] = full_df.censustract.astype(str)

In [23]:
full_data=pd.merge(full_data, boundary_df[['geoid','coords']], left_on='locationname',right_on='geoid', how='left')

In [24]:
full_data.shape

(797, 128)

In [25]:
full_data.head()

,censustract,state_x,county,urban,pop2010,ohu2010,groupquartersflag,numgqtrs,pctgqtrs,lilatracts_1and10,...,stroke among adults aged >=18 years,taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure,visits to dentist or dental clinic among adults aged >=18 years,visits to doctor for routine checkup within the past year among adults aged >=18 years,state_y,fips,mrfei,str_fips,geoid,coords
0,17031010100,Illinois,Cook County,1,4854,2302,0,218.0,4.49,0,...,3.2,70.3,60.4,76.0,IL,1.703101e+10,7.69231,17031010100,17031010100,"[(-87.677199, 42.022942), (-87.670073, 42.0229..."
1,17031010201,Illinois,Cook County,1,6450,2463,0,163.0,2.53,0,...,3.3,67.5,52.7,73.4,NaN,NaN,NaN,NaN,17031010201,"[(-87.684653, 42.019484999999996), (-87.680448..."
2,17031010202,Illinois,Cook County,1,2818,1115,0,315.0,11.18,0,...,3.8,72.4,57.3,75.0,NaN,NaN,NaN,NaN,17031010202,"[(-87.67685499999999, 42.019411), (-87.673388,..."
3,17031010300,Illinois,Cook County,1,6236,2826,0,791.0,12.68,0,...,3.5,73.3,62.8,75.4,IL,1.703101e+10,10.2564,17031010300,17031010300,"[(-87.67133, 42.019374), (-87.66949799999999, ..."
4,17031010400,Illinois,Cook County,1,5042,2098,0,1349.0,26.76,0,...,1.7,62.2,68.7,71.8,IL,1.703101e+10,8.16327,17031010400,17031010400,"[(-87.66345299999999, 42.01283), (-87.66132547..."


In [26]:
list(full_data.columns)

['censustract',
 'state_x',
 'county',
 'urban',
 'pop2010',
 'ohu2010',
 'groupquartersflag',
 'numgqtrs',
 'pctgqtrs',
 'lilatracts_1and10',
 'lilatracts_halfand10',
 'lilatracts_1and20',
 'lilatracts_vehicle',
 'hunvflag',
 'lowincometracts',
 'povertyrate',
 'medianfamilyincome',
 'la1and10',
 'lahalfand10',
 'la1and20',
 'latracts_half',
 'latracts1',
 'latractsvehicle_20',
 'lapop1_10',
 'lapop05_10',
 'lapop1_20',
 'lalowi1_10',
 'lalowi05_10',
 'lalowi1_20',
 'lapophalf',
 'lapophalfshare',
 'lalowihalf',
 'lalowihalfshare',
 'lakidshalf',
 'lakidshalfshare',
 'laseniorshalf',
 'laseniorshalfshare',
 'lawhitehalf',
 'lawhitehalfshare',
 'lablackhalf',
 'lablackhalfshare',
 'laasianhalf',
 'laasianhalfshare',
 'lanhopihalf',
 'lanhopihalfshare',
 'laaianhalf',
 'laaianhalfshare',
 'laomultirhalf',
 'laomultirhalfshare',
 'lahisphalf',
 'lahisphalfshare',
 'lahunvhalf',
 'lahunvhalfshare',
 'lasnaphalf',
 'lasnaphalfshare',
 'lapop1',
 'lapop1share',
 'lalowi1',
 'lalowi1share',


In [27]:
clean_data_path = os.path.join(PROJ_ROOT, 
                         'data',
                         'processed',
                         'full_data.csv')
full_data.to_csv(clean_data_path, index=False)